In [2]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import BeautifulSoup

d:\one drive\OneDrive\Desktop\new-proj\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4362108439")
page_data = loader.load().pop().page_content
print(page_data)









            
          LinkedIn Login, Sign in | LinkedIn
      
        

























































































 









        Sign in
    

 



 








        Sign in with Apple
      










          Sign in with a passkey
      



        By clicking Continue, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy.
    

or











 

          Email or phone
        

 


 












 











 

            Password
        

        Show
      

 


            Forgot password?
          


Keep me logged in



 
              Sign in
          




 



            We’ve emailed a one-time link to your primary email address
        
Click on the link to sign in instantly to your LinkedIn account.







If you don’t see the email in your inbox, check your spam folder.

          Resend email
        

          Back
        





          
    
    

    New to LinkedIn? Join now


In [4]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API"),
    model_name="llama-3.1-8b-instant"
)

if __name__ == "__main__":
    response = llm.invoke("who was the first president of the united states?")
    print(response.content)

The first president of the United States was George Washington. He was inaugurated as the first president in 1789 and served two terms until 1797.


In [17]:
from langchain_core.prompts import PromptTemplate

Prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chin_extract = Prompt_extract | llm
res = chin_extract.invoke(input={'page_data': page_data})
print("Response content:")
print(res.content)
print("\nType:", type(res.content))

Response content:
However, there are no job postings provided in the scraped text. It appears to be a login page for LinkedIn.

If I were to generate a response based on the given text, I would return an empty JSON object:

```
{}
```

But since there's no actual job postings, I'll demonstrate how you could structure a JSON object for a hypothetical job posting:

Assuming the scraped text contains the following job posting:

```
"Software Engineer"
"5-7 years"
"Python, Java, C++, Data Structures"
"This is a software engineer position, responsible for developing and maintaining software systems."
```

You could extract the job posting and return it in JSON format as follows:

```json
{
  "role": "Software Engineer",
  "experience": "5-7 years",
  "skills": "Python, Java, C++, Data Structures",
  "description": "This is a software engineer position, responsible for developing and maintaining software systems."
}
```

Please provide the actual job postings if you'd like me to assist with 

In [18]:
from langchain_core.output_parsers import JsonOutputParser
import json
import re

json_parser = JsonOutputParser()

# Extract JSON from response (finds the last JSON block)
content = res.content
json_blocks = re.findall(r'\{[^{}]*\}', content, re.DOTALL)

if json_blocks:
    # Get the last (most likely) valid JSON block
    for block in reversed(json_blocks):
        try:
            json_res = json.loads(block)
            if json_res:  # If not empty
                break
        except:
            continue
else:
    json_res = {}

print("Parsed JSON:", json_res)

Parsed JSON: {'role': 'Software Engineer', 'experience': '5-7 years', 'skills': 'Python, Java, C++, Data Structures', 'description': 'This is a software engineer position, responsible for developing and maintaining software systems.'}


In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
job = json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [24]:
job

{'role': 'Software Engineer',
 'experience': '5-7 years',
 'skills': 'Python, Java, C++, Data Structures',
 'description': 'This is a software engineer position, responsible for developing and maintaining software systems.'}

In [22]:

job = json_res
job['skills']

'Python, Java, C++, Data Structures'

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Empowering Your Software Development with AtliQ

Dear [Client's Name],

I came across the job description for a Software Engineer position at your organization, and I wanted to introduce you to AtliQ, an AI & Software Consulting company where I work as a Business Development Executive. We specialize in developing tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency for numerous enterprises.

Our expertise in software development aligns perfectly with the requirements mentioned in the job description. With a focus on Python, Java, C++, and data structures, our team of skilled engineers can help you develop and maintain software systems efficiently.

AtliQ has a proven track record of delivering high-quality solutions that meet the unique needs of our clients. I'd like to highlight some of our notable projects:

* Our machine learning portfolio in Python is available at [https://example.com/ml-python-portfolio](https